In [1]:
import openpyxl as op
from tqdm.notebook import tqdm
from django.contrib.admin.utils import flatten
from datetime import datetime

In [2]:
def get_sheet_names(filename):
    wb = op.load_workbook(filename + ".xlsx")
    return wb.sheetnames

In [3]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)

    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [99]:
company = Company.objects.get(name='GES - UHE-MIRANDA')
operational_control = OperationalControl.objects.get(
    firm__company=company,
    kind="707"
)
status = ServiceOrderActionStatus.objects.get(
    companies=company, 
    name="Registro Deferido"
)
firm = Firm.objects.get(
    company=company, 
    name="Administrativo"
)
occ = OccurrenceType.objects.get(
    uuid='71702b2c-0d0c-41c1-b5c4-243a40785967'
)
user=User.objects.get(
    username="engie"
)
st = SearchTag.objects.get(uuid='76159557-8ae4-42b7-8fc8-d16f040b6bb4')
status,firm,occ,user,st,operational_control.uuid

(<ServiceOrderActionStatus: Registro Deferido - OCCURRENCE_RECORD_STATUS>,
 <Firm: [2fd0ca76-b741-47d7-af9c-607e6ed1eca2: GES - UHE-MIRANDA] 9a5dd7fd-d4ce-4e05-ae7d-215e0793db47: Administrativo>,
 <OccurrenceType: Cadastro - ['GES - UHE-PONTE DE PEDRA', 'GES - UHE-MIRANDA', 'GES - UHE-JAGUARA', 'GES - UHE-ITA']>,
 <User: engie - Suporte Kartado>,
 <SearchTag: GES - UHE-MIRANDA - 1 - select_option - Controle Operacional>,
 UUID('fe2b4ffd-19fc-4f7b-bf03-1ac13303db1f'))

In [152]:
filename = '06. UHMI_PI - 2023 - rev marco'
# Abrir a planilha
workbook = op.load_workbook(filename + '.xlsx')
# Selecionar a planilha
sheet = workbook[workbook.sheetnames[4]]
sheet.title,get_sheet_names(filename)

('Proprietários Lindeiros',
 ['Critérios',
  'Associações Locais',
  'Instituições de Ensino',
  'Órgãos e instituições públicas',
  'Proprietários Lindeiros'])

In [153]:
sheet['B3'].value

'PROPRIETÁRIOS LINDEIROS - UHMI'

In [154]:
# Iterar linhas a partir da linha 4, coluna B
dados=[]
cidade=''
obj={}
for i,row in enumerate(sheet.iter_rows(min_row=4, min_col=2),start=3):

    
    if sheet['B'+str(i)].value != "ID" and not isinstance(sheet['B'+str(i)].value,int) and sheet['B'+str(i)].value != None:
        cidade=sheet['B'+str(i)].value.split('/')[0]
        if not obj.get(cidade):
            obj[cidade]=[]
        
    if row[0].value != None and row[0].value != "ID" and i >3 and row[1].value is not None :
        
        obj[cidade].append({'Nome' : row[1].value,
                            'Contato': row[2].value,
                            'Email' : row[3].value,
                            'Telefone' : row[4].value,
                            'Engajamento' : row[5].value,
                            'Postura' : row[6].value,
                            'Data' : row[7].value})
        dados.append(obj)

In [155]:
dados=dados[0]
dados

{'PROPRIETÁRIOS LINDEIROS - UHMI': [{'Nome': 'Geraldo Antônio',
   'Contato': 'Geraldo Antônio2',
   'Email': None,
   'Telefone': '(34) 9 9686-7610',
   'Engajamento': 'Envolvimento',
   'Postura': 'Apoiadora',
   'Data': None},
  {'Nome': 'Valdomiro Fernandes',
   'Contato': 'Valdomiro Fernandes',
   'Email': None,
   'Telefone': '(34) 9 9664-9112',
   'Engajamento': 'Envolvimento',
   'Postura': 'Apoiadora',
   'Data': None},
  {'Nome': 'Ubirajara Peron',
   'Contato': 'Ubirajara Peron',
   'Email': 'bperon@adfert.com.br',
   'Telefone': '(34) 9 9994-5702',
   'Engajamento': 'Consulta / Negociação',
   'Postura': 'Apoiadora',
   'Data': None},
  {'Nome': 'Norvaldo Gonçalves Delgado',
   'Contato': 'Norvaldo Gonçalves Delgado',
   'Email': 'pointdodelgado@yahoo.com.br',
   'Telefone': '(34) 9 9642-5932',
   'Engajamento': 'Colaboração / Parceria / Empoderamento',
   'Postura': 'Apoiadora',
   'Data': None},
  {'Nome': 'Simone do Carmo',
   'Contato': 'Simone do Carmo',
   'Email': No

In [156]:
## Selecionar a Categoria  antes de rodar
options= [{
            "name": "Pessoa física",
            "value": "1"
          },{
            "name": "Associação",
            "value": "2"
          },{
            "name": "Empresa",
            "value": "3"
          },{
            "name": "Grupo organizado",
            "value": "4"
          },{
            "name": "Instituição de ensino",
            "value": "5"
          },{
            "name": "Instituição pública",
            "value": "6"
          },{
            "name": "Outro",
            "value": "7"
          }]
categoria = "1"

In [165]:
recs=[]
for cidade,instituicoes in dados.items():
    print(cidade)
    try:
        city=City.objects.filter(name=cidade.strip().title()).first() if cidade != 'ORGÃOS REGULADORES E ESTRATÉGICOS' and 'PROPRIETÁRIOS LINDEIROS' in cidade else None,
    except:
        print(cidade)
    else:
        for instituicao in tqdm(instituicoes):
            form_data = {
                "category" : categoria,
                'intitution_name' : instituicao['Nome'] if instituicao['Nome'] is not None else '',
                'institutional_representaties':[{'representative_name':instituicao['Contato'],
                                                 'email':instituicao['Email'],
                                                'phone_number':instituicao['Telefone']}],
                'engagement' : get_value('Engajamento',occ,instituicao['Engajamento']) if instituicao['Engajamento'] is not None else "",
                'opinion' : get_value('Postura / Opinião',occ,instituicao['Postura']) if instituicao['Postura'] is not None else "",
                'observation' : str(instituicao['Data']) if instituicao['Data'] is not None else ''
            }
            rec = OccurrenceRecord(
                company = company,
                created_by = user,
                city=city[0] if cidade != 'ORGÃOS REGULADORES E ESTRATÉGICOS' and 'PROPRIETÁRIOS LINDEIROS' in cidade else None,
#                 uf_code = city[0].uf_code if cidade != 'ORGÃOS REGULADORES E ESTRATÉGICOS' and 'PROPRIETÁRIOS LINDEIROS' in cidade else None,
                occurrence_type = occ,
                status = status,
                form_data = form_data,
                firm = firm,
                operational_control = operational_control,
                datetime = datetime.now()
            )
#             rec.search_tags.add(st) Da erro
            recs.append(rec)

PROPRIETÁRIOS LINDEIROS - UHMI


  0%|          | 0/362 [00:00<?, ?it/s]

In [166]:
len(recs)

362

In [170]:
OccurrenceRecord.objects.filter(created_by=user,
                                created_at__gte='2024-3-24',
                                company=company,
                                occurrence_type=occ).count()

544

In [168]:
recs[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7abf4dbd7f40>,
 'uuid': UUID('2162447b-d812-4ac4-9f01-02f90c6c5ed1'),
 'datetime': datetime.datetime(2024, 3, 25, 15, 21, 40, 344614),
 'number': '',
 'created_at': None,
 'updated_at': None,
 'reviews': 0,
 'company_id': UUID('2fd0ca76-b741-47d7-af9c-607e6ed1eca2'),
 'editable': True,
 'is_approved': False,
 'uf_code': None,
 'city_id': None,
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': None,
 'geometry': None,
 'properties': [],
 'distance_from_dam': 0,
 'other_reference': '',
 'origin': None,
 'origin_media': None,
 'informer': {},
 'territorial_administration': '',
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
 'search_tag_description': '',
 'occurrence_type_id': UUID('71702b2c-0d0c-41c1-b5c4-243a40785967'),
 'form_data': {'category': '1',
  'intitution_name': 'Geraldo Antônio',
  'institutional_representaties': [{'representative_name': 'Geraldo Antônio2',
    'email': None,
    'phone_nu

In [169]:
OccurrenceRecord.objects.bulk_create(recs)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-03-25 15:21:40.344614) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-03-25 15:21:40.347846) while time zone support is active.
  warnings.warn("DateTimeField %s received a naive datetime (%s)"
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.8/lib/python3.8/site-packages/django/db/models/fields/__init__.py:1424: RuntimeWarning: DateTimeField OccurrenceRecord.datetime received a naive datetime (2024-03-25 15:21:40.351187) while time zone support is active.
  warnings.warn("DateTimeField %s received 

[<OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.344614>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.347846>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.351187>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.355206>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.358824>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.363488>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.366533>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.372021>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.376653>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.379991>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.383585>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.387208>,
 <OccurrenceRecord: [GES - UHE-MIRANDA] -  - 2024-03-25 15:21:40.390671>,
 <OccurrenceRecord: [GES - UHE-MIRANDA

In [32]:
City.objects.get(name=list(dados.keys())[2].strip().capitalize())

<City: [31] Conquista>

In [31]:
list(dados.keys())[2]

'CONQUISTA'

In [45]:
for a in OccurrenceRecord.objects.filter(operational_control=operational_control,company=company):
    if a.form_data.get('engagement'):
        a.form_data['engagement'] = str(a.form_data.get('engagement'))
    if a.form_data.get('opinion'):
        a.form_data['opinion'] = str(a.form_data.get('opinion'))

{'_state': <django.db.models.base.ModelState at 0x7f07e8275b50>,
 'uuid': UUID('a3bd55b7-e625-44f1-89f7-d271600aa0f7'),
 'datetime': datetime.datetime(2024, 2, 7, 17, 32, 42, 215263),
 'number': '',
 'created_at': datetime.datetime(2024, 2, 7, 20, 33, 30, 967661, tzinfo=<UTC>),
 'updated_at': datetime.datetime(2024, 2, 7, 20, 33, 30, 967686, tzinfo=<UTC>),
 'reviews': 0,
 'company_id': UUID('17663a30-5f71-4e7c-8ac2-adf472342b42'),
 'editable': True,
 'is_approved': False,
 'uf_code': 31,
 'city_id': UUID('60585315-f8c3-4566-8ed6-53ef1798e465'),
 'location_id': None,
 'place_on_dam': '',
 'river_id': None,
 'point': None,
 'geometry': None,
 'properties': [],
 'distance_from_dam': 0,
 'other_reference': '',
 'origin': None,
 'origin_media': None,
 'informer': {},
 'territorial_administration': '',
 'created_by_id': UUID('0aa50773-b368-4a50-9f12-4a7d8dfaf256'),
 'search_tag_description': '',
 'occurrence_type_id': UUID('71702b2c-0d0c-41c1-b5c4-243a40785967'),
 'form_data': {'category': '

In [14]:
ocr=OccurrenceRecord.objects.filter(occurrence_type = occ,
                firm = firm,
                operational_control = operational_control)
ocr.count()

367

In [15]:
for a in tqdm(ocr):
    a.form_data['engagement']=str(a.form_data['engagement'])
    a.form_data['opinion']=str(a.form_data['opinion'])
    a.save()

In [74]:
print('MONTE CARMELO'.title())

Monte Carmelo


In [75]:
city_new = City(uf_code='31',name="Monte Carmelo")

In [76]:
city_new.save()

In [31]:
cidade

'NOVA PONTE'

In [73]:
City.objects.filter(name__icontains='monte')

<QuerySet [<City: [42] Belmonte>, <City: [42] Monte Carlos>, <City: [43] Montenegro>]>